## Chapter 17: Parallel Computing

In [1]:
function countHeads(n::Int)
    c::Int = 0
    for i=1:n
        c += rand(Bool)
    end
    c
end

countHeads (generic function with 1 method)

In [2]:
@time countHeads(2*10^9)/(2*10^9)

 13.781212 seconds (7 allocations: 19.453 KiB)


0.5000011085

In [3]:
using Distributed

In [4]:
addprocs(1)

1-element Array{Int64,1}:
 2

In [5]:
nprocs()

2

In [22]:
@everywhere function countHeads(n::Int)
   c::Int = 0
   for i=1:n
       c += rand(Bool)
   end
   c
end

In [10]:
a= @spawn countHeads(10^9);
b= @spawn countHeads(10^9)

Future(2, 1, 10, nothing)

In [11]:
@time fetch(a)+fetch(b)

 13.402011 seconds (557 allocations: 16.766 KiB)


1000014297

In [12]:
addprocs()

8-element Array{Int64,1}:
  3
  4
  5
  6
  7
  8
  9
 10

In [13]:
@time let
 nheads = @distributed (+) for i = 1:2*10^9
   Int(rand(Bool))
 end
end

  6.666907 seconds (53.46 k allocations: 2.771 MiB)


999984994

In [14]:
include("PlayingCards.jl")
using .PlayingCards, Random

In [15]:
function countHands(trials::Int,f::Function)
    local deck=map(Card,1:52)
    local numhands=0
    for i=1:trials
        shuffle!(deck)
        h = Hand(deck[1:5])
        if(f(h))
            numhands+=1
        end
    end
    numhands
end

countHands (generic function with 1 method)

In [16]:
@time countHands(10_000_000,isFullHouse)

  7.901587 seconds (30.00 M allocations: 3.874 GiB, 4.67% gc time)


14181

In [17]:
@everywhere include("PlayingCards.jl")
@everywhere using .PlayingCards, Random
@everywhere function paraCountHands(trials::Integer,f::Function)
  local deck=map(Card,1:52)
  local numhands=0
  function checkHand(f::Function) ## shuffle the deck then check the hand.
    shuffle!(deck)
    f(Hand(deck[1:5]))
  end
  @distributed (+) for i = 1:trials
    Int(checkHand(f))
  end  
end

In [18]:
@time fh = paraCountHands(10_000_000,isFullHouse)

  3.476821 seconds (330.18 k allocations: 17.387 MiB, 0.14% gc time)


14234

In [19]:
num_coins = 1_000_000_000*ones(Int64,12)

12-element Array{Int64,1}:
 1000000000
 1000000000
 1000000000
 1000000000
 1000000000
 1000000000
 1000000000
 1000000000
 1000000000
 1000000000
 1000000000
 1000000000

In [20]:
@time pmap(countHeads,num_coins)

RemoteException: On worker 6:
UndefVarError: #countHeads not defined
deserialize_datatype at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:1252
handle_deserialize at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:826
deserialize at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:773
handle_deserialize at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:833
deserialize at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:773 [inlined]
deserialize_msg at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Distributed/src/messages.jl:99
#invokelatest#1 at ./essentials.jl:710 [inlined]
invokelatest at ./essentials.jl:709 [inlined]
message_

In [21]:
@time map(countHeads,num_coins)

100.007535 seconds (47.38 k allocations: 2.420 MiB, 0.01% gc time)


12-element Array{Int64,1}:
 500015268
 499973887
 499987114
 499997130
 500001770
 499998898
 500003104
 499993103
 499981763
 500042663
 500000590
 499996257